In [1]:
#import packages
import pandas as pd
import numpy as np
#igniring warnings
import warnings
warnings.filterwarnings('ignore')

C:\Users\asus\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
#read data
import pandas as pd

file_path = r"C:\Users\asus\Downloads\ai_dev_assignment_tickets_complex_1000.xls"
df = pd.read_excel(file_path, engine='xlrd')

In [3]:
df.head()


,ticket_id,ticket_text,issue_type,urgency_level,product
0,1,Payment issue for my SmartWatch V2. I was unde...,Billing Problem,Medium,SmartWatch V2
1,2,Can you tell me more about the UltraClean Vacu...,General Inquiry,NaN,UltraClean Vacuum
2,3,I ordered SoundWave 300 but got EcoBreeze AC i...,Wrong Item,Medium,SoundWave 300
3,4,Facing installation issue with PhotoSnap Cam. ...,Installation Issue,Low,PhotoSnap Cam
4,5,Order #30903 for Vision LED TV is 13 days late...,Late Delivery,NaN,Vision LED TV


In [4]:
#chaeck data And clean data
df.info()
df.head()
print(df['issue_type'].value_counts())
print(df['urgency_level'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ticket_id      1000 non-null   int64 
 1   ticket_text    945 non-null    object
 2   issue_type     924 non-null    object
 3   urgency_level  948 non-null    object
 4   product        1000 non-null   object
dtypes: int64(1), object(4)
memory usage: 39.2+ KB
issue_type
Billing Problem       146
General Inquiry       146
Account Access        143
Installation Issue    142
Product Defect        121
Wrong Item            114
Late Delivery         112
Name: count, dtype: int64
urgency_level
High      330
Medium    319
Low       299
Name: count, dtype: int64


In [5]:
#check missing value
df.isnull().sum()

ticket_id         0
ticket_text      55
issue_type       76
urgency_level    52
product           0
dtype: int64

In [7]:
#cleaning dataa
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

# Download stopwords (run once)
nltk.download('stopwords')

# Load stopwords
stop_words = set(stopwords.words('english'))

# Load data
file_path = r"C:\Users\asus\Downloads\ai_dev_assignment_tickets_complex_1000.xls"
df = pd.read_excel(file_path, engine='xlrd')

# Handle missing data
df = df.dropna(subset=['issue_type', 'urgency_level'])
df['ticket_text'] = df['ticket_text'].fillna('')

# Text Cleaning Function
def clean_text(text):
    text = text.lower()  # Lowercase
    text = re.sub(r'[^a-z\s]', '', text)  # Remove special chars and numbers
    tokens = text.split()  # Tokenize
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
    return " ".join(tokens)

# Apply cleaning
df['clean_text'] = df['ticket_text'].apply(clean_text)

# Check cleaned data
df[['ticket_text', 'clean_text']].head()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,ticket_text,clean_text
0,Payment issue for my SmartWatch V2. I was unde...,payment issue smartwatch v underbilled order
2,I ordered SoundWave 300 but got EcoBreeze AC i...,ordered soundwave got ecobreeze ac instead ord...
3,Facing installation issue with PhotoSnap Cam. ...,facing installation issue photosnap cam setup ...
5,Can you tell me more about the PhotoSnap Cam w...,tell photosnap cam warranty also available red
6,is malfunction. It stopped working after just...,malfunction stopped working days


In [8]:
#Additional Features
from textblob import TextBlob

df['ticket_length'] = df['clean_text'].apply(lambda x: len(x.split()))
df['sentiment'] = df['clean_text'].apply(lambda x: TextBlob(x).sentiment.polarity)


In [9]:
#Text Features
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=1000)
X_tfidf = tfidf.fit_transform(df['clean_text'])


In [10]:
#combine
import numpy as np
from scipy.sparse import hstack
# Combine with additional features
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack

scaler = StandardScaler()
additional_features = scaler.fit_transform(df[['ticket_length', 'sentiment']])
X = hstack([X_tfidf, additional_features])


In [11]:
#Label Encoding
from sklearn.preprocessing import LabelEncoder
le_issue = LabelEncoder()
le_urgency = LabelEncoder()

df['issue_encoded'] = le_issue.fit_transform(df['issue_type'])
df['urgency_encoded'] = le_urgency.fit_transform(df['urgency_level'])




In [12]:
#Model Training
from sklearn.model_selection import train_test_split
X_train, X_test, y_issue_train, y_issue_test = train_test_split(X, df['issue_encoded'], test_size=0.2, random_state=42)
X_train, X_test, y_urgency_train, y_urgency_test = train_test_split(X, df['urgency_encoded'], test_size=0.2, random_state=42)



from sklearn.linear_model import LogisticRegression
issue_model = LogisticRegression(max_iter=1000)
urgency_model = LogisticRegression(max_iter=1000)

issue_model.fit(X_train, y_issue_train)
urgency_model.fit(X_train, y_urgency_train)


LogisticRegression(max_iter=1000)

In [22]:
#Entity Extraction
from sklearn.metrics import classification_report

y_issue_pred = issue_model.predict(X_test)
y_urgency_pred = urgency_model.predict(X_test)

print("Issue Type Classifier:\n", classification_report(y_issue_test, y_issue_pred, target_names=le_issue.classes_))
print("Urgency Classifier:\n", classification_report(y_urgency_test, y_urgency_pred, target_names=le_urgency.classes_))


Issue Type Classifier:
                     precision    recall  f1-score   support

    Account Access       1.00      0.97      0.99        35
   Billing Problem       1.00      0.91      0.95        32
   General Inquiry       0.72      1.00      0.84        29
Installation Issue       1.00      0.88      0.93        24
     Late Delivery       1.00      0.90      0.95        20
    Product Defect       1.00      0.95      0.97        19
        Wrong Item       1.00      0.94      0.97        17

          accuracy                           0.94       176
         macro avg       0.96      0.93      0.94       176
      weighted avg       0.95      0.94      0.94       176

Urgency Classifier:
               precision    recall  f1-score   support

        High       0.35      0.40      0.37        60
         Low       0.32      0.31      0.32        54
      Medium       0.30      0.26      0.28        62

    accuracy                           0.32       176
   macro avg       0

In [13]:
import datefinder

product_list = df['product'].dropna().unique().tolist()
complaint_keywords = ['broken', 'late', 'error', 'issue', 'not working', 'delay', 'crash']

def extract_entities(text):
    entities = {}
    
    # Products
    entities['products'] = [p for p in product_list if p.lower() in text.lower()]
    
    # Dates
    matches = datefinder.find_dates(text)
    entities['dates'] = [str(match.date()) for match in matches]
    
    # Complaint keywords
    entities['complaint_keywords'] = [word for word in complaint_keywords if word in text.lower()]
    
    return entities


In [14]:
#test the entity
sample_text = df['ticket_text'].iloc[0]
extract_entities(sample_text)


{'products': ['SmartWatch V2'], 'dates': [], 'complaint_keywords': ['issue']}

In [15]:
#Integration Function
def analyze_ticket(text):
    clean = clean_text(text)
    tfidf_feat = tfidf.transform([clean])
    add_feat = scaler.transform([[len(clean.split()), TextBlob(clean).sentiment.polarity]])
    X_input = hstack([tfidf_feat, add_feat])
    
    issue_pred = le_issue.inverse_transform(issue_model.predict(X_input))[0]
    urgency_pred = le_urgency.inverse_transform(urgency_model.predict(X_input))[0]
    entities = extract_entities(text)
    
    return {'issue_type': issue_pred, 'urgency_level': urgency_pred, 'entities': entities}

# Example usage:
analyze_ticket("My phone is broken and not working. I purchased it on 25th March.")


{'issue_type': 'Product Defect',
 'urgency_level': 'High',
 'entities': {'products': [],
  'dates': ['2025-03-25'],
  'complaint_keywords': ['broken', 'not working']}}

In [ ]:
#Gradio Interface
import gradio as gr

def gradio_interface(text):
    result = analyze_ticket(text)
    return result

gr.Interface(fn=gradio_interface, inputs=gr.Textbox(label="Ticket Text"), outputs="json").launch()
